# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with:<br>
**mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json**

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(f"MongoDB databases (look for 'uk_food' in this list):\n{mongo.list_database_names()}\n")

MongoDB databases (look for 'uk_food' in this list):
['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'uk_food']



In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(f"Collections in the uk_food database (look for 'establishments' in this list):\n{mongo['uk_food'].list_collection_names()}")

Collections in the uk_food database (look for 'establishments' in this list):
['establishments']


In [6]:
# review a document in the establishments collection
print("Sample document from the establishments collection:\n")
pprint(mongo['uk_food']['establishments'].find_one())

Sample document from the establishments collection:

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65de776267ee65ce5b5ea076'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

print(f"A document for {new_restaurant['BusinessName']} has been inserted into the establishments collection.")

A document for Penang Flavours has been inserted into the establishments collection.


In [10]:
# Check that the new restaurant was inserted
query = {'BusinessName' : new_restaurant['BusinessName']}

print('Here is the document for Penang Flavours:\n')
for doc in establishments.find(query):
    pprint(doc)

Here is the document for Penang Flavours:

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65de776a691a59a79c22082b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType' : new_restaurant['BusinessType']}
fields = {'BusinessTypeID' : 1, 'BusinessType' : 1}

# Run the query and extract the 'BusinessTypeID'
results = establishments.find(query, fields)
new_restaurant_business_type_id = results[0]['BusinessTypeID']

print(f"Here is a document with BusinessType '{new_restaurant['BusinessType']}':\n")
pprint(results[0])

Here is a document with BusinessType 'Restaurant/Cafe/Canteen':

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('65de776267ee65ce5b5ea077')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
query = {'BusinessName' : new_restaurant['BusinessName']}
update = [{'$set' : {'BusinessTypeID' : new_restaurant_business_type_id}}]
establishments.update_one(query, update)

In [13]:
# Confirm that the new restaurant was updated
query = {'BusinessName' : new_restaurant['BusinessName']}

print(f"{new_restaurant['BusinessName']} now has a BusinessTypeID of {new_restaurant_business_type_id}:\n")
for doc in establishments.find(query):
    pprint(doc)

Penang Flavours now has a BusinessTypeID of 1:

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65de776a691a59a79c22082b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName' : 'Dover'}
results = establishments.count_documents(query)

print(f"{results} documents have LocalAuthorityName 'Dover'.")

994 documents have LocalAuthorityName 'Dover'.


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName' : 'Dover'}
establishments.delete_many(query)

print("All documents with LocalAuthorityName 'Dover' have been deleted.")

All documents with LocalAuthorityName 'Dover' have been deleted.


In [16]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName' : 'Dover'}
results = establishments.count_documents(query)

print(f"{results} documents have LocalAuthorityName 'Dover'.")

0 documents have LocalAuthorityName 'Dover'.


In [17]:
# Check that other documents remain with 'find_one'
print("Sample remaining document in the establishments collection:\n")
pprint(establishments.find_one())

Sample remaining document in the establishments collection:

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65de776267ee65ce5b5ea35b'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/10

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
query = {}
update = [{'$set' : {
    'geocode.longitude' : {'$toDouble' : '$geocode.longitude'},
    'geocode.latitude' : {'$toDouble' : '$geocode.latitude'}
}}]

establishments.update_many(query, update)

print('Longitudes and latitudes in the establishments collection now have the type decimal.')

Longitudes and latitudes in the establishments collection now have the type decimal.


Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

print('Rating values in the establishments collection have been cleaned.')

Rating values in the establishments collection have been cleaned.


In [20]:
# Change the data type from String to Integer for RatingValue
query = {}
update = [{'$set' : {'RatingValue' : {'$toInt' : '$RatingValue'}}}]

establishments.update_many(query, update)

print('RatingValue in the establishments collection now has the type integer.')

RatingValue in the establishments collection now has the type integer.


In [21]:
# Check that the coordinates and rating value are now numbers
results = establishments.find_one()

print(f"Longitude data type:   {type(results['geocode']['longitude'])}")
print(f"Latitude data type:    {type(results['geocode']['latitude'])}")
print(f"RatingValue data type: {type(results['RatingValue'])}")

Longitude data type:   <class 'float'>
Latitude data type:    <class 'float'>
RatingValue data type: <class 'int'>
